In [ ]:
import torch
import numpy as np

class EarlyStopping:
    def __init__(self, patience=10, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): Сколько эпох ждать после последнего улучшения метрики
            verbose (bool): Выводить сообщения о каждой проверке (по умолчанию False)
            delta (float): Минимальное изменение для улучшения метрики
            path (str): Путь для сохранения модели
        """
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.path = path
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Сохранение модели, если валидационная потеря уменьшилась'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

# Пример использования

# Инициализация модели, функции потерь и оптимизатора
model = ...  # Ваша модель
criterion = ...  # Ваша функция потерь
optimizer = ...  # Ваш оптимизатор

# Параметры ранней остановки
early_stopping = EarlyStopping(patience=10, verbose=True)

# Цикл обучения
num_epochs = 100
for epoch in range(num_epochs):
    # Обучение
    model.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    # Валидация
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for data, target in val_loader:
            output = model(data)
            loss = criterion(output, target)
            val_loss += loss.item()
    val_loss = val_loss / len(val_loader)

    print(f'Epoch {epoch+1}, Validation Loss: {val_loss:.6f}')

    # Проверка условия ранней остановки
    early_stopping(val_loss, model)

    if early_stopping.early_stop:
        print("Early stopping")
        break

# Загрузка лучших весов модели
model.load_state_dict(torch.load('checkpoint.pt'))


In [ ]:
class CNNLSTM(nn.Module):
    def __init__(self, num_classes):
        super(CNNLSTM, self).__init__()
        self.cnn = models.resnet18(pretrained=True)
        self.cnn = nn.Sequential(*list(self.cnn.children())[:-2])

        self.lstm_input_size = self._get_conv_output_size((3, 224, 224))  # Можно использовать произвольный размер
        self.lstm = nn.LSTM(self.lstm_input_size, 256, batch_first=True)
        self.fc = nn.Linear(256, num_classes)

    def _get_conv_output_size(self, shape):
        o = self.cnn(torch.zeros(1, *shape))
        return int(torch.prod(torch.tensor(o.size())))

    def forward(self, x):
        batch_size, seq_length, c, h, w = x.size()
        x = x.view(batch_size * seq_length, c, h, w)
        x = self.cnn(x)
        x = x.view(batch_size, seq_length, -1)
        x, (h, c) = self.lstm(x)
        x = self.fc(x[:, -1, :])
        return x

model = CNNLSTM(num_classes=10)

# Пример входного тензора с произвольным размером
input_tensor = torch.randn(1, 10, 3, 240, 320)  # [batch_size, seq_length, channels, height, width]
output = model(input_tensor)
print(output.size())  # Ожидаемый размер: [1, 10]
